# 1주차 실습 - 한국어 NLP 기초 소개

2024년 한국어 NLP 및 LLM 입문 과정의 첫 번째 실습 세션에 오신 것을 환영합니다. 이번 실습에서는 한국어 자연어 처리(NLP)의 기본 개념을 탐구하고 몇 가지 기본적인 NLP 작업을 직접 경험해 볼 것입니다.

## 목표

- 한국어 NLP의 기본 개념 이해하기
- Mecab을 사용한 토큰화 및 품사 태깅 구현하기
- Word2Vec을 사용한 한국어 단어 임베딩 시각화하기
- 기본적인 한국어 텍스트 생성 실험하기 (해당되는 경우)

먼저 환경을 설정하고 필요한 라이브러리를 가져오겠습니다.


In [ ]:
# 필요한 라이브러리 설치
%pip install ekonlpy gensim matplotlib scikit-learn openai


In [2]:
# 필요한 모듈 가져오기
import numpy as np
from ekonlpy import Mecab
import gensim.downloader as api
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Mecab 초기화
mecab = Mecab()

print("설정 완료!")

설정 완료!


## 1. 한국어 NLP 기초 소개

자연어 처리(NLP)는 컴퓨터 과학, 인공지능, 언어학이 교차하는 분야입니다. 컴퓨터와 인간 언어 사이의 상호작용에 중점을 둡니다. 한국어 NLP의 주요 개념은 다음과 같습니다:

- 형태소 분석: 한국어 텍스트를 의미 있는 최소 단위(형태소)로 분리하기
- 품사 태깅: 각 형태소에 문법적 범주 할당하기
- 개체명 인식: 텍스트에서 명명된 개체를 식별하고 분류하기
- 단어 임베딩: 한국어 단어의 밀집 벡터 표현

이러한 개념을 보여주기 위해 간단한 예제로 시작해보겠습니다.


In [3]:
# 시연을 위한 샘플 텍스트
sample_text = "금통위는 따라서 물가안정과 병행, 경기상황에 유의하는 금리정책을 펼쳐나가기로 했다고 밝혔다."
print("샘플 텍스트:", sample_text)

샘플 텍스트: 금통위는 따라서 물가안정과 병행, 경기상황에 유의하는 금리정책을 펼쳐나가기로 했다고 밝혔다.


## 2. Mecab을 사용한 형태소 분석 및 품사 태깅 예제

한국어 NLP에서는 형태소 분석과 품사 태깅이 매우 중요합니다. Mecab을 사용하여 이 두 가지 작업을 동시에 수행할 수 있습니다.


In [4]:
def analyze_text(text):
    """Mecab을 사용하여 입력 텍스트의 형태소 분석 및 품사 태깅을 수행합니다."""
    return mecab.pos(text)


# 샘플 텍스트 분석
analysis = analyze_text(sample_text)
print("형태소 분석 및 품사 태깅 결과:")
for morpheme, pos in analysis:
    print(f"{morpheme}: {pos}")

# 형태소와 품사 개수 출력
print(f"\n형태소 개수: {len(analysis)}")
print(f"고유 품사 개수: {len({pos for _, pos in analysis})}")

형태소 분석 및 품사 태깅 결과:
금통위: NNG
는: JX
따라서: MAJ
물가: NNG
안정: NNG
과: JC
병행: NNG
,: SC
경기: NNG
상황: NNG
에: JKB
유의: NNG
하: XSV
는: ETM
금리: NNG
정책: NNG
을: JKO
펼쳐: VV
나가: VX
기: ETN
로: JKB
했: VV
다고: EC
밝혔: VV
다: EF
.: SF

형태소 개수: 26
고유 품사 개수: 15


## 3. 한국어 Word2Vec을 사용한 단어 임베딩 시각화

단어 임베딩은 의미적 관계를 포착하는 단어의 밀집 벡터 표현입니다. 한국어 텍스트로 학습된 Word2Vec 모델을 사용하여 단어 임베딩을 만들고 시각화해보겠습니다.


In [5]:
def visualize_embeddings(model, words):
    """t-SNE를 사용하여 단어 임베딩을 시각화합니다."""
    word_vectors = [model[word] for word in words if word in model]
    if not word_vectors:
        print("모델 어휘에서 단어를 찾을 수 없습니다.")
        return
    word_vectors = np.array(word_vectors)  # numpy 배열로 변환

    # 샘플 수에 따라 perplexity 조정
    n_samples = word_vectors.shape[0]
    perplexity = min(30, n_samples - 1)  # 기본값은 30이지만 n_samples보다 작아야 함

    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity)
    embeddings_2d = tsne.fit_transform(word_vectors)

    plt.figure(figsize=(10, 8))
    for i, word in enumerate(words):
        if word in model:
            x, y = embeddings_2d[i]
            plt.scatter(x, y)
            plt.annotate(word, (x, y))
    plt.title("한국어 단어 임베딩 시각화")
    plt.show()


# 한국어 Word2Vec 모델 로드 (예: 한국어 위키피디아로 학습된 모델)
# 실제 사용 시에는 적절한 한국어 Word2Vec 모델을 로드해야 합니다
# model = Word2Vec.load("path_to_korean_word2vec_model")

# 예제 문장 정의
sentences = [
    "경제 성장률이 예상보다 높게 나타났다",
    "중앙은행이 기준금리를 인상했다",
    "물가상승률이 목표치를 넘어섰다",
    "실업률이 전년 대비 소폭 하락했다",
    "수출 실적이 전월 대비 증가세를 보였다",
]

# 문장에서 모든 고유 형태소 가져오기
all_morphemes = list(
    {morpheme for sentence in sentences for morpheme, _ in mecab.pos(sentence)}
)

# 모델의 어휘에 있는 형태소만 필터링
# words_to_plot = [word for word in all_morphemes if word in model.wv.key_to_index]

# 임베딩 시각화
# visualize_embeddings(model, words_to_plot)

print("한국어 Word2Vec 모델이 필요합니다. 실제 사용 시 적절한 모델을 로드하세요.")

한국어 Word2Vec 모델이 필요합니다. 실제 사용 시 적절한 모델을 로드하세요.


## 4. OpenAI의 GPT 모델을 사용한 기본 한국어 텍스트 생성 (선택 사항)

참고: 이 섹션은 OpenAI API 키가 필요하며 비용이 발생할 수 있습니다. API에 접근할 수 없다면 이 섹션을 건너뛰어도 됩니다.

OpenAI의 GPT 모델을 사용하여 한국어 프롬프트를 기반으로 텍스트를 생성해보겠습니다. 이는 한국어 텍스트 생성 작업에서 대규모 언어 모델의 능력을 보여줍니다.


In [6]:
import openai
import os

# API 키와 모델 이름 설정
MODEL = "gpt-4o-mini"

# OpenAI 클라이언트 초기화
client = openai.OpenAI(
    api_key=os.environ.get(
        "OPENAI_API_KEY",
        "<환경 변수로 설정되지 않은 경우 OpenAI API 키를 여기에 입력하세요>",
    )
)

# 채팅 완성 생성
completion = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "당신은 한국어에 능통한 도우미입니다. 경제 관련 질문에 답변해주세요.",
        },
        {
            "role": "user",
            "content": "한국의 현재 경제 상황에 대해 간단히 설명해주세요.",
        },
    ],
)

# 어시스턴트의 응답 출력
print(f"어시스턴트: {completion.choices[0].message.content}")

어시스턴트: 2023년 한국의 경제 상황은 글로벌 경제의 불확실성과 내부적인 도전 과제가 복합적으로 작용하는 상황입니다. 

1. **성장률**: 한국 경제는 팬데믹 이후 회복세를 보였으나, 글로벌 경기 둔화와 반도체 산업의 부진 등으로 인해 성장률이 둔화되는 추세입니다. 특히 반도체는 한국 경제의 중요한 축인데, 세계 시장의 수요 감소가 영향을 미치고 있습니다.

2. **물가 및 인플레이션**: 물가는 여전히 높은 수준을 유지하고 있으며, 인플레이션 압력이 지속되고 있습니다. 정부는 이러한 물가 상승을 억제하기 위해 금리 인상 등의 통화정책을 시행하고 있습니다.

3. **고용 시장**: 고용 시장은 회복세를 보이고 있으나, 고용의 질과 청년 실업 문제는 여전히 해결해야 할 숙제로 남아 있습니다.

4. **정책 대응**: 정부는 경제 회복 및 지속 가능한 성장을 위해 다양한 정책을 추진하고 있으며, 특히 친환경 산업과 디지털 전환 분야에 대한 투자에 집중하고 있습니다.

전반적으로, 한국 경제는 여러 도전에 직면해 있지만, 구조적인 변화와 혁신을 통해 향후 긍정적인 방향으로 나아갈 것으로 기대됩니다.


## 결론

이번 실습 세션에서 우리는 한국어 자연어 처리의 몇 가지 기본 개념과 기술을 탐구했습니다:

1. Mecab을 사용하여 한국어 형태소 분석과 품사 태깅을 수행했습니다.
2. 한국어 단어 임베딩을 시각화하는 방법을 학습했습니다 (실제 모델 로딩은 필요).
3. (선택적으로) GPT 모델을 사용하여 한국어 텍스트 생성을 실험했습니다.

이러한 기본 기술들은 더 고급 한국어 NLP 작업과 응용 프로그램의 기초를 형성합니다. 앞으로 몇 주 동안, 우리는 이러한 개념들을 바탕으로 더 정교한 한국어 NLP 방법과 모델을 탐구할 것입니다.

## 선택 과제

1. 뉴스 기사나 소설 등 다양한 장르의 한국어 텍스트에 대해 형태소 분석과 품사 태깅을 수행해보고, 결과를 비교해보세요.
2. 한국어 Word2Vec 모델을 직접 학습시켜보세요. 예를 들어, 한국어 위키피디아 데이터를 사용할 수 있습니다.
3. Mecab의 사용자 사전을 활용하여 특정 도메인(예: 경제, 의학)에 특화된 형태소 분석을 수행해보세요.
4. 형태소 분석 결과를 사용하여 간단한 한국어 텍스트 분류 모델을 구현해보세요 (예: 뉴스 기사 주제 분류).
5. 한국어 감성 분석(Sentiment Analysis) 모델을 구현해보세요. 형태소 분석 결과와 머신러닝 알고리즘을 결합하여 사용할 수 있습니다.
